In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers

In [3]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [4]:
from openpyxl import Workbook
from openpyxl import load_workbook


In [5]:
import pathlib
from datetime import datetime

In [6]:
import itertools
import os

In [7]:
(train_ds, validation_ds, test_ds), info = tfds.load(
    "colorectal_histology",
    split=["train[:85%]", "train[85%:95%]", "train[95%:]"],
    with_info=True,
    as_supervised=True,
    shuffle_files= True,
)

num_classes = info.features['label'].num_classes

In [8]:
size = (150, 150)
batch_size = 16

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=1)

In [10]:
%cd  functions

/home/reynier/py_projects/Deployments/functions


In [11]:
from Nets import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
from Simulation import save_obj, load_obj

In [12]:
model = GetNeuralNetworkModel('ResNet',(150,150,3),8, quantization = False, aging_active=False)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 150, 150, 3)  0           ['input_1[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 150, 150, 3)  0           ['rescaling[0][0]']              
                                                                                                  
 lambda_1 (Lambda)              (None, 150, 150, 3)  0           ['lambda[0][0]']             

                                                                                                  
 lambda_11 (Lambda)             (None, 37, 37, 64)   0           ['activation_5[0][0]']           
                                                                                                  
 conv2d_7 (Conv2D)              (None, 37, 37, 256)  16640       ['lambda_11[0][0]']              
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 37, 37, 256)  1024       ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_6 (Activation)      (None, 37, 37, 256)  0           ['batch_normalization_7[0][0]']  
                                                                                                  
 lambda_12

 ormalization)                                                                                    
                                                                                                  
 add_3 (Add)                    (None, 19, 19, 512)  0           ['batch_normalization_13[0][0]', 
                                                                  'batch_normalization_14[0][0]'] 
                                                                                                  
 activation_14 (Activation)     (None, 19, 19, 512)  0           ['add_3[0][0]']                  
                                                                                                  
 lambda_22 (Lambda)             (None, 19, 19, 512)  0           ['activation_14[0][0]']          
                                                                                                  
 lambda_23 (Lambda)             (None, 19, 19, 512)  0           ['lambda_22[0][0]']              
          

                                                                                                  
 activation_23 (Activation)     (None, 19, 19, 128)  0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 lambda_32 (Lambda)             (None, 19, 19, 128)  0           ['activation_23[0][0]']          
                                                                                                  
 conv2d_22 (Conv2D)             (None, 19, 19, 128)  147584      ['lambda_32[0][0]']              
                                                                                                  
 batch_normalization_22 (BatchN  (None, 19, 19, 128)  512        ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 lambda_42 (Lambda)             (None, 10, 10, 256)  0           ['activation_30[0][0]']          
                                                                                                  
 conv2d_29 (Conv2D)             (None, 10, 10, 256)  590080      ['lambda_42[0][0]']              
                                                                                                  
 batch_normalization_29 (BatchN  (None, 10, 10, 256)  1024       ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_31 (Activation)     (None, 10, 10, 256)  0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 lambda_43 (Lambda)             (None, 10, 10, 256)  0           ['activation_31[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_39 (Activation)     (None, 10, 10, 256)  0           ['batch_normalization_35[0][0]'] 
                                                                                                  
 lambda_51 (Lambda)             (None, 10, 10, 256)  0           ['activation_39[0][0]']          
                                                                                                  
 conv2d_36 (Conv2D)             (None, 10, 10, 1024  263168      ['lambda_51[0][0]']              
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 10, 10, 1024  4096       ['conv2d_36[0][0]']              
 ormalizat

                                                                                                  
 conv2d_42 (Conv2D)             (None, 10, 10, 1024  263168      ['lambda_59[0][0]']              
                                )                                                                 
                                                                                                  
 batch_normalization_42 (BatchN  (None, 10, 10, 1024  4096       ['conv2d_42[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_48 (Activation)     (None, 10, 10, 1024  0           ['batch_normalization_42[0][0]'] 
                                )                                                                 
                                                                                                  
 lambda_60

 batch_normalization_49 (BatchN  (None, 5, 5, 2048)  8192        ['conv2d_49[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 5, 5, 2048)   0           ['batch_normalization_49[0][0]'] 
                                                                                                  
 lambda_70 (Lambda)             (None, 5, 5, 2048)   0           ['activation_55[0][0]']          
                                                                                                  
 add_14 (Add)                   (None, 5, 5, 2048)   0           ['lambda_70[0][0]',              
                                                                  'lambda_67[0][0]']              
                                                                                                  
 activatio

In [15]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [16]:
%cd  ../ResNet50/Weights/

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [17]:
model.load_weights('weights.data')

In [18]:
score = model.evaluate(train_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

266/266 [==============================] - 22s 73ms/step - loss: 0.3339 - accuracy: 0.9174
Test loss: 0.33387869596481323
Test accuracy: 0.917411744594574


In [20]:
for index,layer in enumerate(model.layers):
    print(index,layer.name)

0 input_1
1 rescaling
2 lambda
3 lambda_1
4 zero_padding2d
5 conv2d
6 batch_normalization
7 activation
8 lambda_2
9 lambda_3
10 max_pooling2d
11 conv2d_1
12 batch_normalization_1
13 activation_1
14 lambda_4
15 lambda_5
16 conv2d_2
17 batch_normalization_2
18 activation_2
19 lambda_6
20 lambda_7
21 conv2d_3
22 conv2d_4
23 batch_normalization_3
24 batch_normalization_4
25 add
26 activation_3
27 lambda_8
28 lambda_9
29 conv2d_5
30 batch_normalization_5
31 activation_4
32 lambda_10
33 conv2d_6
34 batch_normalization_6
35 activation_5
36 lambda_11
37 conv2d_7
38 batch_normalization_7
39 activation_6
40 lambda_12
41 add_1
42 activation_7
43 lambda_13
44 conv2d_8
45 batch_normalization_8
46 activation_8
47 lambda_14
48 conv2d_9
49 batch_normalization_9
50 activation_9
51 lambda_15
52 conv2d_10
53 batch_normalization_10
54 activation_10
55 lambda_16
56 add_2
57 activation_11
58 lambda_17
59 conv2d_11
60 batch_normalization_11
61 activation_12
62 lambda_18
63 lambda_19
64 conv2d_12
65 batch_nor

In [1]:
Indices = [0,5,10,11,14,17,18,23,26,29,34,37,40,45,48,51,52,57,60,63,68,71,74,79,82,85,90,93,102,105,108,
 113,116,119,127,130,135,138,141,146,149,152,160,163,164,169,172,175,180,183,186,191]

In [2]:
len(Indices)

52

In [19]:
num_address  = 2359808


Indices      = [5,10,11,16,21,22,29,33,37,44,48,52,59,64,69,70,77,81,85,92,96,100,107,111,115,122,
                127,132,133,140,144,148,155,159,163,170,174,178,185,189,193,200,204,208,215,220,225,
                226,233,237,241,248,252,256,263,268]
                
samples      = 150

Data         = GetReadAndWrites(model,Indices,num_address,samples,CNN_gating=False)
stats        = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)

procesados:  25
procesados:  50
procesados:  75
procesados:  100
procesados:  125
procesados:  150


In [20]:
Accs_x= []
network_size   = 290400*16   # Tamaño del buffer (en bits)
num_of_samples = 10       # Numero de muestras de distintas configuraciones de fallos a testear por cada valor de Accs/Loss

n_bits_total = np.ceil(network_size).astype(int)    #numero de bits totales

buffer = np.array(['x']*(network_size))
print(buffer)
for index in range(0,num_of_samples):
  loss,acc_x   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3), 
                                      act_frac_size = 12, act_int_size = 3, wgt_frac_size = 13, wgt_int_size = 2, 
                                      batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False)
  print(acc_x)
  Accs_x.append(acc_x)
print(str(n_bits_total)+' completada: ', datetime.now().strftime("%H:%M:%S"))

['x' 'x' 'x' ... 'x' 'x' 'x']
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 75ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [==============================] - 1s 76ms/step - loss: 0.6760 - accuracy: 0.7800
0.7799999713897705
16/16 [

In [21]:
Accs = []
Accs_w = []
Accs_a_w = []
Accs_1 = []
Accs_w_1 = []
Accs_a_w_1 = []
buffer_size= 16777216
Accs_1_0 = []
Accs_w_1_0 = []
Accs_a_w_1_0 = []
buffer_size= 16777216

In [22]:
import pathlib
from datetime import datetime

In [23]:
ruta_bin = '../../Data/Fault_Characterization/VC707/RawData/'
directorio = pathlib.Path(ruta_bin)

ficheros = [fichero.name for fichero in directorio.iterdir()]

In [24]:
for base, dirs, files in os.walk(ruta_bin):
  print(files)
  
  for i, j in enumerate(files):
    directorio = os.path.join(ruta_bin, j)
    print(directorio)
    buffer = (analize_file(directorio, buffer_size))
    error_mask, locs = (buffer_vectores(buffer))
 
    loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3), 
                                      act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1, 
                                      batch_size=16, verbose = 0, aging_active = True, weights_faults = False,
                                      faulty_addresses = locs, masked_faults = error_mask)
    loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                        act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                        batch_size=16, verbose = 0, aging_active = False, weights_faults = True,
                                        faulty_addresses = locs, masked_faults = error_mask)
    
    loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                          act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                          batch_size=16, verbose = 0, aging_active = True, weights_faults = True,
                                          faulty_addresses = locs, masked_faults = error_mask)

    Accs.append(acc)
    #Accs_w.append(acc_w)
    #Accs_a_w.append(acc_a_w)
    
    print('acc_0:', Accs)
    print('acc_w0:',Accs_w)
    print('acc_a_w_0:',Accs_a_w)
    
Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

Acc.head()

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))

['VC707-0.54.bin', 'VC707-0.55.bin', 'VC707-0.56.bin', 'VC707-0.57.bin', 'VC707-0.60.bin', 'VC707-0.59.bin', 'VC707-0.58.bin']
../../Data/Fault_Characterization/VC707/RawData/VC707-0.54.bin
soy el buffer dentro de funciones 16777216
 dentro de funciones Cantidad de elementos por tipo : Counter({'x': 16766856, '0': 10360})
acc_0: [0.800000011920929]
acc_w0: []
acc_a_w_0: []
../../Data/Fault_Characterization/VC707/RawData/VC707-0.55.bin
soy el buffer dentro de funciones 16777216
 dentro de funciones Cantidad de elementos por tipo : Counter({'x': 16773858, '0': 3358})
acc_0: [0.800000011920929, 0.8080000281333923]
acc_w0: []
acc_a_w_0: []
../../Data/Fault_Characterization/VC707/RawData/VC707-0.56.bin
soy el buffer dentro de funciones 16777216
 dentro de funciones Cantidad de elementos por tipo : Counter({'x': 16776066, '0': 1150})
acc_0: [0.800000011920929, 0.8080000281333923, 0.8119999766349792]
acc_w0: []
acc_a_w_0: []
../../Data/Fault_Characterization/VC707/RawData/VC707-0.57.bin
soy e

In [25]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [26]:
locs = load_obj('locs_054')
error_mask = load_obj('error_mask_054')

In [27]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [28]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_054.xlsx', sheet_name='fichero_707_054', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:44:13
 completada:  15:44:13


In [29]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [30]:
locs = load_obj('locs_055')
error_mask = load_obj('error_mask_055')

In [31]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [32]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_055.xlsx', sheet_name='fichero_707_055', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:44:40
 completada:  15:44:40


In [33]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [34]:
locs = load_obj('locs_056')
error_mask = load_obj('error_mask_056')

In [35]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [36]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_056.xlsx', sheet_name='fichero_707_056', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:44:59
 completada:  15:44:59


In [37]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [38]:
locs = load_obj('locs_057')
error_mask = load_obj('error_mask_057')

In [39]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [40]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_057.xlsx', sheet_name='fichero_707_055', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:45:16
 completada:  15:45:16


In [41]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [42]:
locs = load_obj('locs_058')
error_mask = load_obj('error_mask_058')

In [43]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [44]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_058.xlsx', sheet_name='fichero_707_058', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:45:29
 completada:  15:45:29


In [45]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [46]:
locs = load_obj('locs_059')
error_mask = load_obj('error_mask_059')

In [47]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [48]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_059.xlsx', sheet_name='fichero_707_059', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:45:40
 completada:  15:45:40


In [49]:
%cd '../../Data/error_mask_x/VC_707'

/home/reynier/py_projects/Deployments/Data/error_mask_x/VC_707


In [50]:
locs = load_obj('locs_060')
error_mask = load_obj('error_mask_060')

In [51]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [52]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../error_mask_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_060.xlsx', sheet_name='fichero_707_060', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/error_mask_x
 completada:  15:45:51
 completada:  15:45:51


In [53]:
%cd '../../Data/mask_volteada/VC_707'

/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707


In [54]:
locs = load_obj('locs_054')
error_mask = load_obj('error_mask_054')

In [55]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [56]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../mask_volteada/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_054.xlsx', sheet_name='fichero_707_054', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/mask_volteada
 completada:  15:46:37
 completada:  15:46:37


In [57]:
%cd '../../Data/byte_x/VC_707'

/home/reynier/py_projects/Deployments/Data/byte_x/VC_707


In [58]:
locs = load_obj('locs_054')
error_mask = load_obj('error_mask_054')

In [59]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [60]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

loss,acc_a_w   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)

Accs.append(acc)
Accs_w.append(acc_w)
Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)

%cd  '../byte_x/'

buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
buf_cero.columns =['Acc', 'A_w', 'Acc_a_w']
buf_cero.to_excel('ResNet_fichero_fichero_707_054.xlsx', sheet_name='fichero_707_054', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))


/home/reynier/py_projects/Deployments/ResNet50/byte_x
 completada:  15:47:23
 completada:  15:47:23


In [61]:
%cd '../../Data/byte_x/VC_707'

/home/reynier/py_projects/Deployments/Data/byte_x/VC_707


In [62]:
%cd  '../../../ResNet50/Weights/'

/home/reynier/py_projects/Deployments/ResNet50/Weights


In [63]:
Accs = []

vol=54
for i in range(7):
    %cd '../../Data/byte_x/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../ResNet50/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    print(acc)
    
Acc_ResNet=pd.DataFrame(Accs)

buf_cero = pd.concat([Acc_ResNet], axis=1, join='outer')
buf_cero.columns =['Acc']
buf_cero.to_excel('ResNet_fichero_fichero_Flip-patch.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/byte_x/VC_707
tamaño de locs 2
/home/reynier/py_projects/

In [64]:
Accs = []

vol=54
for i in range(7):
    %cd '../../Data/ECC/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../ResNet50/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    print(acc)
    
Acc_ResNet=pd.DataFrame(Accs)

buf_cero = pd.concat([Acc_ResNet], axis=1, join='outer')
buf_cero.columns =['Acc']
buf_cero.to_excel('ResNet_fichero_fichero_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.828000009059906
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8159999847412109
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECC/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/ResNet5

In [65]:
Accs = []

vol=54
for i in range(7):
    %cd '../../Data/ECCx/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../ResNet50/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    print(acc)
    
Acc_ResNet=pd.DataFrame(Accs)

buf_cero = pd.concat([Acc_ResNet], axis=1, join='outer')
buf_cero.columns =['Acc']
buf_cero.to_excel('ResNet_fichero_fichero_Iso-Area_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.6399999856948853
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.7039999961853027
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8159999847412109
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/ECCx/VC_707
tamaño de locs 2
/home/reynier/py_projects/Deployments/Re

In [66]:
Accs = []

vol=54
for i in range(7):
    %cd '../../Data/mask_volteada/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../ResNet50/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('ResNet', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 14, wgt_int_size = 1,
                                            batch_size=batch_size, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    print(acc)
    
Acc_ResNet=pd.DataFrame(Accs)

buf_cero = pd.concat([Acc_ResNet], axis=1, join='outer')
buf_cero.columns =['Acc']
buf_cero.to_excel('ResNet_fichero_fichero_mask_volteada.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 9439
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 3191
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 1126
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 337
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 56
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/mask_volteada/VC_707
tamaño de locs 6
/home/reynier/py_projects/Deployments/ResNet50/Weights
0.8119999766349792
/home/reynier/py_projects/Deployments/Data/mask_volteada/V